In [1]:
from parse import *
import pandas as pd
%matplotlib inline

In [234]:
import string
table = str.maketrans({key: None for key in string.punctuation})

def remove_punctuation(text):
    return text.translate(table)

In [2]:
G = parseText('data/electionActLabeled.txt', verbose=False)

node: Alberta_Election_Act:{}
node: Alberta_Election_Act--None--0--None--0--Interpretation:{'text': 'Interpretation', 'level': 6}
node: Alberta_Election_Act--None--0--None--0--Interpretation--1--1:{'text': 'In this Act,', 'level': 8}
node: Alberta_Election_Act--None--0--None--0--Interpretation--1--1--a:{'text': '“advance poll” means a poll taken in advance of polling day;', 'level': 9}
node: Alberta_Election_Act--None--0--None--0--Interpretation--1--1--b:{'text': '“by‑election” means an election other than a general election;', 'level': 9}
node: Alberta_Election_Act--None--0--None--0--Interpretation--1--1--b.1:{'text': '“campaign period” means', 'level': 9}
node: Alberta_Election_Act--None--0--None--0--Interpretation--1--1--b.1--i:{'text': 'in the case of a general election held in accordance with section 38.1(2), the period commencing on February 1 in the year in which the election is held and ending 2 months after polling day,', 'level': 10}
node: Alberta_Election_Act--None--0--None-

In [3]:

G.out_edges('Alberta_Election_Act--None--0--None--0--Interpretation')

OutEdgeDataView([('Alberta_Election_Act--None--0--None--0--Interpretation', 'Alberta_Election_Act--None--0--None--0--Interpretation--1--1'), ('Alberta_Election_Act--None--0--None--0--Interpretation', 'Alberta_Election_Act--None--0--None--0--Interpretation--1--2'), ('Alberta_Election_Act--None--0--None--0--Interpretation', 'Alberta_Election_Act--None--0--None--0--Interpretation--1--2.1'), ('Alberta_Election_Act--None--0--None--0--Interpretation', 'Alberta_Election_Act--None--0--None--0--Interpretation--1--2.2'), ('Alberta_Election_Act--None--0--None--0--Interpretation', 'Alberta_Election_Act--None--0--None--0--Interpretation--1--2.3'), ('Alberta_Election_Act--None--0--None--0--Interpretation', 'Alberta_Election_Act--None--0--None--0--Interpretation--1--2.4'), ('Alberta_Election_Act--None--0--None--0--Interpretation', 'Alberta_Election_Act--None--0--None--0--Interpretation--1--3'), ('Alberta_Election_Act--None--0--None--0--Interpretation', 'Alberta_Election_Act--None--0--None--0--Interpr

In [4]:
interp = set(nx.dfs_tree(G, source='Alberta_Election_Act--None--0--None--0--Interpretation'))
#G.remove_node('Alberta_Election_Act--None--0--None--0--Interpretation')
rem = set(nx.dfs_tree(G, source='Alberta_Election_Act').nodes)
print(len(interp))
print(len(rem))
candidates = rem-interp

83
1981


In [5]:
from sklearn.model_selection import train_test_split as tts
x_train, x_test = tts(list(candidates), test_size=0.10, random_state=42)

In [6]:
print(len(x_train))

1708


In [10]:
#print(G[0]['text'])
for node in G:
    print(node, G.nodes[node])

Alberta_Election_Act {}
Alberta_Election_Act--None--0--None--0--Interpretation {'text': 'Interpretation', 'level': 6}
Alberta_Election_Act--None--0--None--0--Interpretation--1--1 {'text': 'In this Act,', 'level': 8}
Alberta_Election_Act--None--0--None--0--Interpretation--1--1--a {'text': '“advance poll” means a poll taken in advance of polling day;', 'level': 9}
Alberta_Election_Act--None--0--None--0--Interpretation--1--1--b {'text': '“by‑election” means an election other than a general election;', 'level': 9}
Alberta_Election_Act--None--0--None--0--Interpretation--1--1--b.1 {'text': '“campaign period” means', 'level': 9}
Alberta_Election_Act--None--0--None--0--Interpretation--1--1--b.1--i {'text': 'in the case of a general election held in accordance with section 38.1(2), the period commencing on February 1 in the year in which the election is held and ending 2 months after polling day,', 'level': 10}
Alberta_Election_Act--None--0--None--0--Interpretation--1--1--b.1--ii {'text': 'in t

In [14]:
df = pd.DataFrame({'text':[G.nodes[x]['text'] for  x in x_train if len(G.nodes[x])>0]})
df['level'] = [G.nodes[x]['level'] for  x in x_train if len(G.nodes[x])>0]
df['node'] = [x for  x in x_train if len(G.nodes[x])>0]
#df.head()
df.to_csv('train.csv')
df.head()

#.to_csv)('train.csv')

,text,level,node
0,The Chief Electoral Officer shall from time to...,8,Alberta_Election_Act--Appointments--1--None--0...
1,Identification documents,6,Alberta_Election_Act--Election_Lists--2--None-...
2,The returning officer shall,8,Alberta_Election_Act--Post‑Polling‑Day_Procedu...
3,if no candidate can be declared elected becaus...,9,Alberta_Election_Act--Post‑Polling‑Day_Procedu...
4,number each objection to a ballot in the poll ...,9,Alberta_Election_Act--Post‑Polling‑Day_Procedu...


In [15]:
len(df)

1707

# evaluation functions

In [236]:
def entities_from_tags(text, tags):
    entities = []
    in_ent =False
    for te, ta in zip(text.split(), tags):
        #print(te, ta)
        try:
            if ta == 'U':
                entities.append(te)
            if ta == 'B':
                entities.append([te])
            if ta == 'I' or ta == 'L':
                entities[-1].append(te)
            if ta == 'L':
                entities[-1] = ' '.join(entities[-1])
        except:
            print(text)
            print(tags)
    return entities
def fix_tags(break_index, tags):
    if break_index:
        return tags[1:]
    return tags
    #print(break_index)
def to_set(entities):
    s = set()
    for ent in entities:
        s.update(ent.split())
    return s

In [237]:
from itertools import chain

eval_data = pd.read_csv('annotated.csv')
eval_data = eval_data.loc[:, ~eval_data.columns.str.contains('^Unnamed')]
eval_data.rename(columns={'0':'text'}, inplace=True)
#df.Tags.apply(lambda x: x[1:-1].split(','))
eval_data['tags'] = eval_data['tags'].apply(lambda x: list(map(lambda y: str(y).strip().strip("'"), x[1:-1].split(','))))
eval_data['tags'] = eval_data.apply(lambda x: fix_tags(getBreak(x['text']), x['tags']), axis=1)
eval_data['text'] = eval_data['text'].apply(lambda x: remove_punctuation(x[getBreak(x):].strip()))
eval_data['entities'] = eval_data.apply( lambda x: entities_from_tags(x['text'], x['tags']), axis = 1)
eval_data['entity_indexes'] = eval_data.tags.apply(lambda x: [i for i,y in enumerate(x) if y in "BILU"])
eval_data['entity_token_set'] = eval_data.entities.apply(to_set)
eval_data['entity_set'] = eval_data.entities.apply(lambda x: set(x))
eval_data.to_csv('final_evaluation_data.csv')
eval_data.head()

,text,tags,entities,entity_indexes,entity_token_set,entity_set
0,a statement of the availability of barrier‑fre...,"[O, U, O, O, O, O, B, L, O, O, O, O, O, B, L, ...","[statement, barrier‑free accessibility, return...","[1, 6, 7, 13, 14, 18, 19, 20]","{officer, polling, statement, accessibility, b...","{advance polling places, statement, returning ..."
1,The following restrictions apply with respect ...,"[O, O, O, O, O, O, O, O, O, O, B, I, L]",[accessible voting equipment],"[10, 11, 12]","{equipment, accessible, voting}",{accessible voting equipment}
2,On the coming into force of subsection 1 the f...,"[O, O, O, O, O, O, B, L, O, O, O]",[subsection 1],"[6, 7]","{subsection, 1}",{subsection 1}
3,are Canadian citizens,"[O, B, L]",[Canadian citizens],"[1, 2]","{Canadian, citizens}",{Canadian citizens}
4,forfeits the person’s right to vote in the ele...,"[O, O, O, O, O, O, O, O, U, O]",[election],[8],{election},{election}


In [238]:
def evaluate_entity_sets(entity_sets):
    results = []
    total_correctly_matched = 0
    total_false_positives = 0
    total_false_negatives = 0
    
    for s, v in zip(entity_sets, eval_data.entity_set):
        correctly_matched = len(s.intersection(v))
        false_positive = len(s-v)
        false_negative = len(v-s)
        total_correctly_matched += correctly_matched
        total_false_positives += false_positive
        total_false_negatives += false_negative
        results.append({'correct':correctly_matched, 'false_positive':false_positive, 'false_negative':false_negative})
    precision = total_correctly_matched/(total_correctly_matched+total_false_positives)
    recall = total_correctly_matched/(total_correctly_matched+total_false_negatives)
    f1 = 2*((precision*recall)/(precision+recall))
    return {'precision':precision, 'recall':recall, 'f1':f1, 'results':results}

def evaluate_entity_token_sets(entity_sets):
    results = []
    total_correctly_matched = 0
    total_false_positives = 0
    total_false_negatives = 0
    
    for s, v in zip(entity_sets, eval_data.entity_token_set):
        correctly_matched = len(s.intersection(v))
        false_positive = len(s-v)
        false_negative = len(v-s)
        total_correctly_matched += correctly_matched
        total_false_positives += false_positive
        total_false_negatives += false_negative
        results.append({'correct':correctly_matched, 'false_positive':false_positive, 'false_negative':false_negative})
    precision = total_correctly_matched/(total_correctly_matched+total_false_positives)
    recall = total_correctly_matched/(total_correctly_matched+total_false_negatives)
    f1 = 2*((precision*recall)/(precision+recall))
    return {'precision':precision, 'recall':recall, 'f1':f1, 'results':results}

round1 = evaluate_entity_token_sets(eval_data.entity_token_set)
assert round1['f1']==1.0
assert round1['recall']==1.0
assert round1['precision']==1.0

round1 = evaluate_entity_sets(eval_data.entity_set)
assert round1['f1']==1.0
assert round1['recall']==1.0
assert round1['precision']==1.0


# prepare the data

In [239]:
import spacy

nlp = spacy.load("en_core_web_lg")

# for token in doc:
#     print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
#             token.shape_, token.is_alpha, token.is_stop)
def spacyize(text):
    spac = nlp(text)
    return {'lemmas':[token.lemma_ for token in spac], 'pos':[token.pos_ for token in spac], 'stops':[token.is_stop for token in spac]}
#print(spacyize("Apple is looking at buying U.K. startup for $1 billion"))


In [240]:
spac = eval_data.text.apply(spacyize)
eval_data['lemmas'] = [x['lemmas'] for x in spac]
eval_data['pos'] = [x['pos'] for x in spac]
eval_data['stops'] = [x['stops'] for x in spac]

In [241]:
eval_data.head()

,text,tags,entities,entity_indexes,entity_token_set,entity_set,lemmas,pos,stops
0,a statement of the availability of barrier‑fre...,"[O, U, O, O, O, O, B, L, O, O, O, O, O, B, L, ...","[statement, barrier‑free accessibility, return...","[1, 6, 7, 13, 14, 18, 19, 20]","{officer, polling, statement, accessibility, b...","{advance polling places, statement, returning ...","[a, statement, of, the, availability, of, barr...","[DET, NOUN, ADP, DET, NOUN, ADP, NUM, NOUN, AD...","[True, False, True, True, False, True, False, ..."
1,The following restrictions apply with respect ...,"[O, O, O, O, O, O, O, O, O, O, B, I, L]",[accessible voting equipment],"[10, 11, 12]","{equipment, accessible, voting}",{accessible voting equipment},"[the, follow, restriction, apply, with, respec...","[DET, VERB, NOUN, VERB, ADP, NOUN, ADP, DET, N...","[True, False, False, False, True, False, True,..."
2,On the coming into force of subsection 1 the f...,"[O, O, O, O, O, O, B, L, O, O, O]",[subsection 1],"[6, 7]","{subsection, 1}",{subsection 1},"[on, the, come, into, force, of, subsection, 1...","[ADP, DET, VERB, ADP, NOUN, ADP, NOUN, NUM, DE...","[True, True, False, True, False, True, False, ..."
3,are Canadian citizens,"[O, B, L]",[Canadian citizens],"[1, 2]","{Canadian, citizens}",{Canadian citizens},"[be, canadian, citizen]","[AUX, ADJ, NOUN]","[True, False, False]"
4,forfeits the person’s right to vote in the ele...,"[O, O, O, O, O, O, O, O, U, O]",[election],[8],{election},{election},"[forfeit, the, person, ’s, right, to, vote, in...","[VERB, DET, NOUN, PART, NOUN, PART, VERB, ADP,...","[False, True, False, True, False, True, False,..."


In [242]:
spac = df.text.apply(spacyize)
df['lemmas'] = [x['lemmas'] for x in spac]
df['pos'] = [x['pos'] for x in spac]
df['stops'] = [x['stops'] for x in spac]

In [243]:
df.head()

,text,level,node,lemmas,pos,stops
0,The Chief Electoral Officer shall from time to...,8,Alberta_Election_Act--Appointments--1--None--0...,"[the, Chief, Electoral, Officer, shall, from, ...","[DET, PROPN, PROPN, PROPN, AUX, ADP, NOUN, ADP...","[True, False, False, False, False, True, False..."
1,Identification documents,6,Alberta_Election_Act--Election_Lists--2--None-...,"[identification, document]","[NOUN, NOUN]","[False, False]"
2,The returning officer shall,8,Alberta_Election_Act--Post‑Polling‑Day_Procedu...,"[the, return, officer, shall]","[DET, VERB, NOUN, AUX]","[True, False, False, False]"
3,if no candidate can be declared elected becaus...,9,Alberta_Election_Act--Post‑Polling‑Day_Procedu...,"[if, no, candidate, can, be, declare, elect, b...","[SCONJ, DET, NOUN, AUX, AUX, VERB, VERB, SCONJ...","[True, True, False, True, True, False, False, ..."
4,number each objection to a ballot in the poll ...,9,Alberta_Election_Act--Post‑Polling‑Day_Procedu...,"[number, each, objection, to, a, ballot, in, t...","[NOUN, DET, NOUN, ADP, DET, NOUN, ADP, DET, NO...","[False, True, False, True, True, False, True, ..."


In [244]:
df.to_csv('dataset/train.csv')
eval_data.to_csv('dataset/test.csv')

In [245]:
interpretations = pd.DataFrame({'text':[G.nodes[x]['text'].strip() for  x in interp if len(G.nodes[x])>0]})
interpretations['level'] = [G.nodes[x]['level'] for  x in interp if len(G.nodes[x])>0]
interpretations['node'] = [x for  x in interp if len(G.nodes[x])>0]
interpretations

,text,level,node
0,“Special Ballot” means the Special Ballot prov...,9,Alberta_Election_Act--None--0--None--0--Interp...
1,"any facility not referred to in subclause (i),...",10,Alberta_Election_Act--None--0--None--0--Interp...
2,“poll book” means a poll book referred to in s...,9,Alberta_Election_Act--None--0--None--0--Interp...
3,“register” means the register of electors esta...,9,Alberta_Election_Act--None--0--None--0--Interp...
4,“oath” includes an affirmation;,9,Alberta_Election_Act--None--0--None--0--Interp...
...,...,...,...
78,in the case of an election under the Alberta S...,10,Alberta_Election_Act--None--0--None--0--Interp...
79,his or her residence before being incarcerated;,9,Alberta_Election_Act--None--0--None--0--Interp...
80,when a person leaves Alberta with the intentio...,9,Alberta_Election_Act--None--0--None--0--Interp...
81,“election” means an election of a person as a ...,9,Alberta_Election_Act--None--0--None--0--Interp...


In [358]:
import re

seed_finder = re.compile(r'“.+”')

def get_seeds(text):
    m = seed_finder.match(text)
    if m:
        return m.group(0).strip('“”')
interpretations['seed'] = interpretations.text.apply(get_seeds )

In [362]:
import tensorflow as tf

import tensorflow_hub as hub

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)


module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [451]:
def evaluate_entity_sets(entity_sets):
    results = []
    total_correctly_matched = 0
    total_false_positives = 0
    total_false_negatives = 0
    
    for s, v in zip(entity_sets, eval_data.entity_set):
        correctly_matched = len(s.intersection(v))
        false_positive = len(s-v)
        false_negative = len(v-s)
        total_correctly_matched += correctly_matched
        total_false_positives += false_positive
        total_false_negatives += false_negative
        results.append({'correct':correctly_matched, 'false_positive':false_positive, 'false_negative':false_negative})
    precision = total_correctly_matched/(total_correctly_matched+total_false_positives)
    recall = total_correctly_matched/(total_correctly_matched+total_false_negatives)
    f1 = 2*((precision*recall)/(precision+recall))
    return {'precision':precision, 'recall':recall, 'f1':f1, 'results':results}

def evaluate_entity_token_sets(entity_sets):
    results = []
    total_correctly_matched = 0
    total_false_positives = 0
    total_false_negatives = 0
    
    for s, v in zip(entity_sets, eval_data.entity_token_set):
        correctly_matched = len(s.intersection(v))
        false_positive = len(s-v)
        false_negative = len(v-s)
        total_correctly_matched += correctly_matched
        total_false_positives += false_positive
        total_false_negatives += false_negative
        results.append({'correct':correctly_matched, 'false_positive':false_positive, 'false_negative':false_negative})
    precision = total_correctly_matched/(total_correctly_matched+total_false_positives)
    recall = total_correctly_matched/(total_correctly_matched+total_false_negatives)
    f1 = 2*((precision*recall)/(precision+recall))
    return {'precision':precision, 'recall':recall, 'f1':f1, 'results':results}

class Seed():
    def __init__(self, text):
        self.text = text.lower()
        self.re = re.compile(f"\\b{self.text}\\b", re.I)
        self.embedding = embed([self.text])
        spac = spacyize(seed['text'])
        self.lemmas = spac['lemmas']
        self.pos = spac['pos']
        self.stops = spac['stops']
    def __repr__(self):
        return self.text
    def __str__(self):
        return self.__repr__()

class SeedList():
    def __init__(self, startList):
        self.seeds = [Seed(text) for text in startList ]
        self.sort()
    def add(self, text, sort = True):
        self.seeds.append(Seed(text))
        if sort:
            self.sort()
    def sort(self):
        self.seeds.sort(key=lambda x: len(x.text), reverse = True)
    def __len__(self):
        return len(self.seeds)
    
    def get_from_seeds(self, text, whole=True):
        ents = set()
        marked = [-1]*len(text)
        for i, seed in enumerate(self.seeds):
            for m in seed.re.finditer(text):
                #print(m.start(), m.end(), m.group())
                for x in range(m.start(), m.end()):
                    if marked[x]!=-1:break
                else:
                    for x in range(m.start(), m.end()):
                        marked[x]=i
                    if whole:
                        ents.add(seed.text)
                    else:
                        ents.update(seed.text.split())
        return ents
    
    def score(self, text):
        emb = embed([text])
        sc = (0.0, '')
        for seed in self.seeds:
            this_score = np.inner(seed.embedding, emb)
            if this_score>sc[0]:
                sc = (float(this_score), seed.text)
        return sc

        
class Patterns():
    def __init__(self, startList, evaluation_data):
        self.seedList = SeedList(startList)
        self.eval_data = eval_data
        
    def print_seeds(self):
        for x in self.seedList.seeds:
            print(x)
            
    def __len__(self):
        return len(self.seedList)
    
    def check_seeds(self, text, whole=True):
        return self.seedList.get_from_seeds(text, whole=whole)

    def add_seed(self, text):
        self.seedList.add(text)
    
    def score(text):
        return self.seedList.score(text)

    
score('Election commish')
    def evaluate(self):
        initial_seed_set = self.eval_data.text.apply(seeds.check_seeds)
        r = evaluate_entity_sets(initial_seed_set)
        print('Results for whole entitiy objects')
        print(f"Entity Set Precision: {r['precision']}")
        print(f"Entity Set Recall: {r['recall']}")
        print(f"Entity Set F1: {r['f1']}")
        print()

        print('Results for partial entitiy objects')
        initial_token_set = self.eval_data.text.apply(seeds.check_seeds, whole=False)
        rtok = evaluate_entity_token_sets(initial_token_set)
        print(f"Entity Token Set Precision: {rtok['precision']}")
        print(f"Entity Token Set Recall: {rtok['recall']}")
        print(f"Entity Token Set F1: {rtok['f1']}")
        
        return r, rtok

    

IndentationError: unexpected indent (<ipython-input-451-f28ee284cb22>, line 116)

In [452]:
seeds = Patterns(set((x for x in interpretations.seed if x)), eval_data)
seeds.print_seeds()

registered constituency association
registered political party
supportive living facility
community support centre
chief financial officer
election commissioner
special mobile poll
polling subdivision
standing committee
electoral division
licensed premises
returning officer
emergency shelter
list of electors
general election
election officer
treatment centre
corrupt practice
rejected ballot
declined ballot
campaign period
polling station
spoiled ballot
official agent
special ballot
recorded mail
polling place
advance poll
by‑election
enumeration
mobile poll
polling day
subdivision
declaration
respondent
scrutineer
enumerator
candidate
poll book
election
register
elector
member
voting
court
clerk
judge
writ
oath


In [453]:
len(seeds)

49

In [454]:
seeds.check_seeds('Election Finances and Contributions Disclosure Act')

{'election'}

In [455]:
seeds.check_seeds('Election Finances and Contributions Disclosure Act', whole=False)

{'election'}

# initial results on seed dict from interpretation entities :)

In [456]:
seeds.evaluate()

Results for whole entitiy objects
Entity Set Precision: 0.6451612903225806
Entity Set Recall: 0.2872531418312388
Entity Set F1: 0.39751552795031053

Results for partial entitiy objects
Entity Token Set Precision: 0.8384615384615385
Entity Token Set Recall: 0.3199608610567515
Entity Token Set F1: 0.463172804532578


({'precision': 0.6451612903225806,
  'recall': 0.2872531418312388,
  'f1': 0.39751552795031053,
  'results': [{'correct': 1, 'false_positive': 0, 'false_negative': 3},
   {'correct': 0, 'false_positive': 1, 'false_negative': 1},
   {'correct': 0, 'false_positive': 0, 'false_negative': 1},
   {'correct': 0, 'false_positive': 0, 'false_negative': 1},
   {'correct': 1, 'false_positive': 0, 'false_negative': 0},
   {'correct': 1, 'false_positive': 0, 'false_negative': 5},
   {'correct': 1, 'false_positive': 0, 'false_negative': 0},
   {'correct': 0, 'false_positive': 0, 'false_negative': 3},
   {'correct': 0, 'false_positive': 0, 'false_negative': 1},
   {'correct': 0, 'false_positive': 0, 'false_negative': 0},
   {'correct': 0, 'false_positive': 1, 'false_negative': 1},
   {'correct': 0, 'false_positive': 0, 'false_negative': 1},
   {'correct': 1, 'false_positive': 0, 'false_negative': 4},
   {'correct': 0, 'false_positive': 0, 'false_negative': 0},
   {'correct': 0, 'false_positive': 0, 

# Spacy Evaluation

In [457]:
part_results = []
whole_results = []

for text in eval_data.text:
    wholes = set()
    parts = set()
    
    doc = nlp(text)
    for ent in doc.ents:
        wholes.add(ent.text)
        parts.update(ent.text.split())
    part_results.append(parts)
    whole_results.append(wholes)
#print(part_results)
#print()
#print(whole_results)
r = evaluate_entity_sets(whole_results)
print(f"Entity Set Precision: {r['precision']}")
print(f"Entity Set Recall: {r['recall']}")
print(f"Entity Set F1: {r['f1']}")
print()
rtok = evaluate_entity_token_sets(part_results)
print(f"Entity Token Set Precision: {rtok['precision']}")
print(f"Entity Token Set Recall: {rtok['recall']}")
print(f"Entity Token Set F1: {rtok['f1']}")

Entity Set Precision: 0.2824427480916031
Entity Set Recall: 0.06642728904847396
Entity Set F1: 0.1075581395348837

Entity Token Set Precision: 0.7834645669291339
Entity Token Set Recall: 0.19471624266144813
Entity Token Set F1: 0.31191222570532917


# seed expansion

In [458]:
import numpy as np
a, b = embed(['the cat in the hat', 'the cat on the mat'])
c, d = embed(['the house', 'the home'])

In [459]:
np.inner(a,b)

0.59086883

In [460]:
np.inner(c,d)

0.9123461

In [461]:

seeds.score('Election commish')
        

AttributeError: 'Patterns' object has no attribute 'score'

In [462]:
seeds

In [ ]:
def score(text):
    emb = embed([text])
    sc = (0.0, '')
    for seed in seeds:
        this_score = np.inner(seed['embedding'],emb)
        if this_score>sc[0]:
            sc = (float(this_score), seed['text'])
    return sc
score('Election commish')

In [352]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

s = embed(['Election commish'])[0].reshape(1,-1)
print(s.shape)
print(seed_df['embedding'].to_numpy().shape)
np.dot(s, seed_df['embedding'].to_numpy())

(1, 512)
(49,)


ValueError: shapes (1,512) and (49,) not aligned: 512 (dim 1) != 49 (dim 0)

In [354]:
np.tensordot(s, seed_df['embedding'].to_numpy(), axes=0)

array([[[<tf.Tensor: shape=(512,), dtype=float32, numpy=
         array([-2.67855846e-03,  1.41176500e-03,  1.16799085e-03, -8.08450684e-04,
                 1.29212299e-03, -1.44443742e-03,  3.07135261e-03,  2.66714534e-03,
                 9.07349808e-04,  2.94826576e-03,  3.44156957e-04, -2.43021478e-03,
                -2.17619396e-04, -1.58651615e-03, -1.55468856e-03,  1.55299983e-03,
                 4.00890160e-04,  1.69853296e-03,  1.56060071e-03, -1.02220300e-04,
                -7.79698661e-04, -1.69823738e-03, -2.22696154e-03, -1.97131746e-03,
                 3.62570601e-04,  7.57100410e-04, -3.66110791e-04, -1.74389780e-03,
                 3.17340554e-03, -2.54110550e-04,  1.61948998e-03, -5.30718011e-04,
                -1.47010258e-03,  2.88391975e-03,  7.19866133e-04,  2.51876586e-03,
                 2.22838979e-04, -1.19266251e-05,  1.82359165e-03, -2.83378572e-03,
                 2.95843068e-03, -1.44436039e-04,  2.02957354e-03, -1.74343644e-03,
                 1.